In [1]:
import os
import torch
import transformers
from tqdm import tqdm
from vllm import LLM, SamplingParams
from transformers import AutoModel, AutoTokenizer
from huggingface_hub import login

/burg/dsi/users/nd2794/conda_envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-30 03:01:57,557	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


ERROR 07-30 03:02:09 pynccl.py:53] Failed to load NCCL library from libnccl.so.2 .It is expected if you are not running on NVIDIA/AMD GPUs.Otherwise please set the environment variable VLLM_NCCL_SO_PATH to point to the correct nccl library path.
INFO 07-30 03:02:09 pynccl_utils.py:17] Failed to import NCCL library: libnccl.so.2: cannot open shared object file: No such file or directory
INFO 07-30 03:02:09 pynccl_utils.py:18] It is expected if you are not running on NVIDIA GPUs.


In [ ]:
cache_dir = '/dvmm-filer3a/users/nrdyava/hf_home'
os.environ['HF_HOME'] = cache_dir
hf_token = !cat $HF_HOME/huggingface/token